## global configuration

In [29]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

device = "cpu"

WINDOW_LSTM = 10
TRAIN_START = 1982
TRAIN_END   = 2013
VAL_START   = 2014
VAL_END     = 2024
ALPHA = 0.10


## Dataset import and Feature engineering

In [30]:
df = pd.read_csv("pop1982-2024_adj.csv")

In [31]:
import numpy as np

# Encode province / region IDs
prov_map = {v: i for i, v in enumerate(df["COD_PRO"].unique())}
reg_map  = {v: i for i, v in enumerate(df["COD_REG"].unique())}

df["PROV_ID"] = df["COD_PRO"].map(prov_map)
df["REG_ID"]  = df["COD_REG"].map(reg_map)

# Log population and growth
df = df.sort_values(["COD_COM", "ANNO"])

# --- MODIFIED SECTION ---
epsilon = 1e-6  # Small value to prevent log(0) or log(NaN)

# 1. fillna(0) turns NAs into 0
# 2. clip(lower=epsilon) turns 0 (and the previous NAs) into epsilon
df["LOG_POP"] = np.log(df["POPOLAZIONE"].fillna(0).clip(lower=epsilon))
# ------------------------

df["DLOG_POP"] = df.groupby("COD_COM")["LOG_POP"].diff()

## LSTM dataset

In [32]:
class LSTMDataset(torch.utils.data.Dataset):
    def __init__(self, df, window, start_year, end_year):
        self.samples = []

        for _, g in df.groupby("COD_COM"):
            g = g.sort_values("ANNO")
            g = g[(g["ANNO"] >= start_year) & (g["ANNO"] <= end_year)]
            if len(g) <= window:
                continue

            y = g["DLOG_POP"].values
            p = int(g["PROV_ID"].iloc[0])
            r = int(g["REG_ID"].iloc[0])

            for i in range(window, len(y)):
                x_win = y[i-window:i]
                y_t = y[i]

                if np.isnan(x_win).any() or np.isnan(y_t):
                    continue

                if np.isinf(x_win).any() or np.isinf(y_t):
                    continue

                self.samples.append((x_win, y_t, p, r))


    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        x, y, p, r = self.samples[idx]
        return (
            torch.nan_to_num(torch.tensor(x, dtype=torch.float32)).unsqueeze(-1),
            torch.nan_to_num(torch.tensor(y, dtype=torch.float32)),
            torch.tensor(p, dtype=torch.long),
            torch.tensor(r, dtype=torch.long),
        )



## Population LSTM (point predictor) and training

In [33]:
class PopulationLSTM(nn.Module):
    def __init__(self, n_prov, n_reg, emb=8, hidden=64, layers=2):
        super().__init__()
        self.ep = nn.Embedding(n_prov, emb)
        self.er = nn.Embedding(n_reg, emb)

        self.lstm = nn.LSTM(
            input_size=1 + 2*emb,
            hidden_size=hidden,
            num_layers=layers,
            batch_first=True
        )
        self.out = nn.Linear(hidden, 1)

    def forward(self, x, p, r):
        ep = self.ep(p).unsqueeze(1).expand(-1, x.size(1), -1)
        er = self.er(r).unsqueeze(1).expand(-1, x.size(1), -1)
        z = torch.cat([x, ep, er], dim=-1)
        h, _ = self.lstm(z)
        return self.out(h[:, -1]).squeeze(-1)


In [34]:
train_ds = LSTMDataset(df, WINDOW_LSTM, TRAIN_START, TRAIN_END)
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)

model_lstm = PopulationLSTM(len(prov_map), len(reg_map)).to(device)
opt = torch.optim.Adam(model_lstm.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()



In [35]:
for epoch in range(10):
    total = 0.0
    for x, y, p, r in train_loader:
        x, y, p, r = x.to(device), y.to(device), p.to(device), r.to(device)
        opt.zero_grad()
        yhat = model_lstm(x, p, r)
        loss = loss_fn(yhat, y)
        loss.backward()
        opt.step()
        total += loss.item()
    print(f"Epoch {epoch+1}: loss={total/len(train_loader):.5f}")

Epoch 1: loss=0.00038
Epoch 2: loss=0.00035
Epoch 3: loss=0.00034
Epoch 4: loss=0.00034
Epoch 5: loss=0.00034
Epoch 6: loss=0.00033
Epoch 7: loss=0.00033
Epoch 8: loss=0.00033
Epoch 9: loss=0.00033
Epoch 10: loss=0.00033


## Sequential EnbPI ensemble and training

In [36]:
class EnsemblePopulationLSTM(nn.Module):
    def __init__(self, base_model_fn, n_models=10):
        super().__init__()
        self.models = nn.ModuleList([base_model_fn() for _ in range(n_models)])
        self.inbag_sets = []

    def predict(self, x, p, r):
        preds = torch.stack([m(x, p, r) for m in self.models])
        return preds.mean(dim=0)

    def predict_oob(self, x, p, r, cod_com):
        outs = []
        for m, inbag in zip(self.models, self.inbag_sets):
            if cod_com not in inbag:
                outs.append(m(x, p, r))
        if len(outs) == 0:
            outs = [m(x, p, r) for m in self.models]
        return torch.stack(outs).mean(dim=0)


In [37]:
def make_population_lstm():
    return PopulationLSTM(len(prov_map), len(reg_map))


In [38]:
def train_ensemble_lstm_sequential(
    df,
    base_model_fn,
    window_lstm,
    train_start,
    train_end,
    n_models=10,
    epochs=10,
):
    df_train = df[(df["ANNO"] >= train_start) & (df["ANNO"] <= train_end)]
    munis = df_train["COD_COM"].unique()

    ensemble = EnsemblePopulationLSTM(base_model_fn, n_models)

    # Progress bar over ensemble members
    for b in tqdm(range(n_models), desc="Training ensemble models"):
        boot = np.random.choice(munis, size=len(munis), replace=True)
        ensemble.inbag_sets.append(set(boot))

        df_boot = pd.concat([df_train[df_train["COD_COM"] == m] for m in boot])
        train_ds = LSTMDataset(df_boot, window_lstm, train_start, train_end)
        loader = DataLoader(train_ds, batch_size=128, shuffle=True)

        model = ensemble.models[b]
        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        criterion = nn.MSELoss()

        # Progress bar over epochs (nested)
        for _ in tqdm(range(epochs), desc=f"Model {b+1}/{n_models}", leave=False):
            for x, y, p, r in loader:
                opt.zero_grad()
                loss = criterion(model(x, p, r), y)
                loss.backward()
                opt.step()

    return ensemble


In [39]:
ensemble_lstm = train_ensemble_lstm_sequential(
    df,
    make_population_lstm,
    WINDOW_LSTM,
    TRAIN_START,
    TRAIN_END,
)


Training ensemble models: 100%|██████████| 10/10 [28:34<00:00, 171.49s/it]


## BUILD LOO RESIDUALS 

In [40]:
@torch.no_grad()
def build_loo_residuals(df, ensemble, window_lstm, start_year, end_year):
    rows = []
    df_hist = df[(df["ANNO"] >= start_year) & (df["ANNO"] <= end_year)]

    # Progress bar over municipalities
    for cod, g in tqdm(
        df_hist.groupby("COD_COM"),
        desc="Computing LOO residuals",
        total=df_hist["COD_COM"].nunique(),
    ):
        g = g.sort_values("ANNO")
        if len(g) <= window_lstm:
            continue

        y = g["DLOG_POP"].values
        logp = g["LOG_POP"].values
        p = torch.tensor([g["PROV_ID"].iloc[0]])
        r = torch.tensor([g["REG_ID"].iloc[0]])

        for i in range(window_lstm, len(y)):
            x = (
                torch.tensor(y[i - window_lstm : i])
                .float()
                .unsqueeze(0)
                .unsqueeze(-1)
            )
            yhat = ensemble.predict_oob(x, p, r, cod).item()

            rows.append(
                {
                    "COD_COM": cod,
                    "ANNO": int(g.iloc[i]["ANNO"]),
                    "EPS": y[i] - yhat,
                    "LOG_POP": logp[i],
                    "PROV_ID": int(p),
                    "REG_ID": int(r),
                }
            )

    return pd.DataFrame(rows)


In [41]:
df_res_loo = build_loo_residuals(
    df, ensemble_lstm, WINDOW_LSTM, TRAIN_START, TRAIN_END
)


Computing LOO residuals: 100%|██████████| 8873/8873 [05:30<00:00, 26.88it/s] 


## SANITY CHECK FOR RESIDUALS

In [42]:
assert not df_res_loo.empty
print(df_res_loo.head())
print("Residuals:", len(df_res_loo))
print("Municipalities:", df_res_loo["COD_COM"].nunique())


   COD_COM  ANNO       EPS   LOG_POP  PROV_ID  REG_ID
0     1001  1992       NaN  7.873978        0       0
1     1001  1993 -0.012765  7.862497        0       0
2     1001  1994  0.015309  7.874739        0       0
3     1001  1995 -0.005263  7.879291        0       0
4     1001  1996 -0.003825  7.878913        0       0
Residuals: 168347
Municipalities: 8419


## Residual Quantile Transformer and TRAINING

In [43]:
qs_all = np.array([
    0.01, 0.025, 0.05,
    0.10, 0.20, 0.30,
    0.40, 0.50, 0.60,
    0.70, 0.80, 0.90,
    0.95, 0.975, 0.99
])


In [44]:
class ResidualDataset(torch.utils.data.Dataset):
    def __init__(self, df_res, window):
        self.samples = []

        for _, g in df_res.groupby("COD_COM"):
            g = g.sort_values("ANNO")
            if len(g) <= window:
                continue

            eps = g["EPS"].values
            logp = g["LOG_POP"].values
            p = int(g["PROV_ID"].iloc[0])
            r = int(g["REG_ID"].iloc[0])

            for i in range(window, len(eps)):
                self.samples.append((
                    eps[i-window:i],
                    logp[i-window:i],
                    eps[i],
                    p,
                    r
                ))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        e, lp, y, p, r = self.samples[idx]
        return (
            torch.tensor(e, dtype=torch.float32),
            torch.tensor(lp, dtype=torch.float32),
            torch.tensor(y, dtype=torch.float32),
            torch.tensor(p, dtype=torch.long),
            torch.tensor(r, dtype=torch.long),
        )


In [45]:
class ResidualQuantileTransformer(nn.Module):
    def __init__(self, n_prov, n_reg, n_q, emb=8, hidden=64):
        super().__init__()
        self.ep = nn.Embedding(n_prov, emb)
        self.er = nn.Embedding(n_reg, emb)

        self.net = nn.Sequential(
            nn.Linear(2 + emb*2, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, n_q)
        )

    def forward(self, eps_hist, logp_hist, p, r):
        e_mean = eps_hist.mean(dim=1, keepdim=True)
        lp_last = logp_hist[:, -1:].clone()

        ep = self.ep(p)
        er = self.er(r)

        x = torch.cat([e_mean, lp_last, ep, er], dim=1)
        return self.net(x)


In [46]:
def multi_quantile_loss(y, yhat, qs):
    loss = 0.0
    for i, q in enumerate(qs):
        e = y - yhat[:, i]
        loss += torch.mean(torch.maximum(q*e, (q-1)*e))
    return loss


In [59]:
WINDOW_RES = 10

df_res_loo = df_res_loo.replace([np.inf, -np.inf], np.nan)
df_res_loo = df_res_loo.dropna(subset=["EPS", "LOG_POP"])


res_ds = ResidualDataset(df_res_loo, WINDOW_RES)
res_loader = DataLoader(res_ds, batch_size=256, shuffle=True)

res_model = ResidualQuantileTransformer(
    n_prov=len(prov_map),
    n_reg=len(reg_map),
    n_q=len(qs_all)
).to(device)

opt = torch.optim.Adam(res_model.parameters(), lr=1e-3)

for epoch in range(30):
    tot = 0.0
    for eps, logp, y, p, r in res_loader:
        eps, logp = eps.to(device), logp.to(device)
        y, p, r = y.to(device), p.to(device), r.to(device)

        opt.zero_grad()
        yhat = res_model(eps, logp, p, r)
        loss = multi_quantile_loss(y, yhat, qs_all)
        loss.backward()
        opt.step()

        tot += loss.item()

    print(f"Residual model epoch {epoch+1}: loss={tot/len(res_loader):.4f}")


Residual model epoch 1: loss=0.0846
Residual model epoch 2: loss=0.0541
Residual model epoch 3: loss=0.0527
Residual model epoch 4: loss=0.0521
Residual model epoch 5: loss=0.0519
Residual model epoch 6: loss=0.0516
Residual model epoch 7: loss=0.0515
Residual model epoch 8: loss=0.0514
Residual model epoch 9: loss=0.0512
Residual model epoch 10: loss=0.0511
Residual model epoch 11: loss=0.0511
Residual model epoch 12: loss=0.0509
Residual model epoch 13: loss=0.0509
Residual model epoch 14: loss=0.0508
Residual model epoch 15: loss=0.0508
Residual model epoch 16: loss=0.0507
Residual model epoch 17: loss=0.0507
Residual model epoch 18: loss=0.0506
Residual model epoch 19: loss=0.0506
Residual model epoch 20: loss=0.0506
Residual model epoch 21: loss=0.0505
Residual model epoch 22: loss=0.0505
Residual model epoch 23: loss=0.0505
Residual model epoch 24: loss=0.0504
Residual model epoch 25: loss=0.0504
Residual model epoch 26: loss=0.0504
Residual model epoch 27: loss=0.0504
Residual m

## Residual quantile helpers

In [60]:
@torch.no_grad()
def residual_quantiles(
    res_model,
    eps_hist,
    logp_hist,
    prov_id,
    reg_id,
):
    e = torch.tensor(eps_hist).float().unsqueeze(0)
    lp = torch.tensor(logp_hist).float().unsqueeze(0)
    p = torch.tensor([prov_id])
    r = torch.tensor([reg_id])

    qhat = res_model(e, lp, p, r).cpu().numpy()[0]
    return dict(zip(qs_all, qhat))


## AgACI validation (adaptive $\alpha$)

In [61]:
def agaci_update(alpha_t, err, alpha_target=0.10, gamma=0.05):
    return np.clip(alpha_t + gamma*(alpha_target - err), 0.01, 0.5)


In [62]:
def validate_agaci(
    df,
    ensemble,
    res_model,
    df_res_hist,
    window_lstm,
    window_res,
    val_start,
    val_end,
    alpha_target=0.10,
    gamma=0.05,
):
    alpha_t = alpha_target
    rows = []

    # --- FIX START ---
    # Include the lookback window in the data selection.
    # We need 'window_lstm' years prior to 'val_start' to form the first input sequence.
    history_start = val_start - window_lstm
    df_val = df[(df["ANNO"] >= history_start) & (df["ANNO"] <= val_end)]
    # --- FIX END ---

    eps_by_com = df_res_hist.groupby("COD_COM")["EPS"].apply(list)

    for cod, g in df_val.groupby("COD_COM"):
        g = g.sort_values("ANNO")
        
        # Now len(g) should be (val_end - val_start + 1) + window_lstm
        if len(g) <= window_lstm:
            continue

        p = int(g["PROV_ID"].iloc[0])
        r = int(g["REG_ID"].iloc[0])

        # Initialize histories with the lookback period
        dlog_hist = list(g["DLOG_POP"].values[:window_lstm])
        logp_hist = list(g["LOG_POP"].values[:window_res])
        eps_hist = eps_by_com.get(cod, [0.0]*window_res)[-window_res:]

        # The last known log_pop before the validation period starts
        log_pop = g["LOG_POP"].iloc[window_lstm-1]

        # Loop starts at window_lstm, which corresponds exactly to 'val_start'
        for i in range(window_lstm, len(g)):
            x = torch.tensor(dlog_hist).float().unsqueeze(0).unsqueeze(-1)
            # Use appropriate device if needed, e.g. x.to(device)
            # Assuming models are on CPU for this snippet or handled internally
            yhat = ensemble.predict(x, torch.tensor([p]), torch.tensor([r])).item()

            qdict = residual_quantiles(
                res_model, eps_hist, logp_hist, p, r
            )

            qlo = qdict[min(qdict, key=lambda q: abs(q - alpha_t/2))]
            qhi = qdict[min(qdict, key=lambda q: abs(q - (1-alpha_t/2)))]

            log_pop = log_pop + yhat
            y_true = g["POPOLAZIONE"].iloc[i]

            lo = np.exp(log_pop + qlo)
            hi = np.exp(log_pop + qhi)

            err = int(not (lo <= y_true <= hi))
            alpha_t = agaci_update(alpha_t, err, alpha_target, gamma)

            rows.append({
                "COD_COM": cod,
                "ANNO": g["ANNO"].iloc[i],
                "ERR": err,
                "ALPHA": alpha_t,
            })

            dlog_hist.append(yhat); dlog_hist.pop(0)
            eps_hist.append(0.0); eps_hist.pop(0)
            logp_hist.append(log_pop); logp_hist.pop(0)

    return pd.DataFrame(rows)

In [64]:
agaci_df = validate_agaci(
    df,
    ensemble_lstm,
    res_model,
    df_res_loo,
    WINDOW_LSTM,
    WINDOW_RES,
    VAL_START,
    VAL_END,
)


In [65]:
print("Mean coverage:", 1 - agaci_df["ERR"].mean())
print("Mean alpha:", agaci_df["ALPHA"].mean())


Mean coverage: 0.41788530301641147
Mean alpha: 0.018401026863482172


## Train a region-level LSTM 

In [76]:
df_reg = (
    df.groupby(["COD_REG", "ANNO"], as_index=False)["POPOLAZIONE"]
      .sum()
      .sort_values(["COD_REG", "ANNO"])
      .copy()
)

df_reg["LOG_POP_REG"] = np.log(df_reg["POPOLAZIONE"].clip(lower=1))
# 1. Calculate the difference (creates NaNs for the first year)
df_reg["DLOG_POP_REG"] = df_reg.groupby("COD_REG")["LOG_POP_REG"].diff()

# 2. DROP the NaN rows
df_reg = df_reg.dropna(subset=["DLOG_POP_REG"]).copy()

# 3. Verify clean-up
print(f"Remaining NaNs: {df_reg['DLOG_POP_REG'].isna().sum()}")


Remaining NaNs: 0


In [77]:
import torch.nn.functional as F

class RegionLSTMDataset(torch.utils.data.Dataset):
    def __init__(self, df_reg, window, start_year, end_year):
        self.samples = []
        for rid, g in df_reg.groupby("COD_REG"):
            g = g.sort_values("ANNO")
            g = g[(g["ANNO"] >= start_year) & (g["ANNO"] <= end_year)]
            if len(g) <= window:
                continue
            y = g["DLOG_POP_REG"].values
            for i in range(window, len(y)):
                self.samples.append((rid, y[i-window:i], y[i]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        rid, x, y = self.samples[idx]
        return (
            torch.tensor([rid], dtype=torch.long),   # region code (we will remap)
            torch.tensor(x, dtype=torch.float32).unsqueeze(-1),
            torch.tensor(y, dtype=torch.float32),
        )

class RegionLSTM(nn.Module):
    def __init__(self, n_reg, emb=8, hidden=64, layers=2):
        super().__init__()
        # Embedding layer for Region ID
        self.er = nn.Embedding(n_reg, emb)
        
        # LSTM input size = 1 (population value) + emb (region vector size)
        self.lstm = nn.LSTM(input_size=1+emb, hidden_size=hidden, num_layers=layers, batch_first=True)
        
        # Output layer (renamed to 'fc' to match your forward code)
        self.fc = nn.Linear(hidden, 3)

    def forward(self, rid, x):
        # --- 1. Prepare Region Embeddings ---
        # rid shape: (batch_size)
        # r_emb shape: (batch_size, emb_dim)
        r_emb = self.er(rid) 
        
        # Repeat the embedding for every time step in the sequence
        # x shape: (batch_size, seq_len, 1)
        seq_len = x.size(1)
        
        # Expand r_emb to (batch_size, seq_len, emb_dim)
        r_emb_expanded = r_emb.unsqueeze(1).repeat(1, seq_len, 1)
        
        # --- 2. Concatenate Features ---
        # Combine [Population Value] + [Region Embedding] at every step
        # combined shape: (batch_size, seq_len, 1 + emb_dim)
        x_combined = torch.cat([x, r_emb_expanded], dim=2)

        # --- 3. LSTM Processing ---
        lstm_out, _ = self.lstm(x_combined)
        
        # Take the output of the last time step
        last_step = lstm_out[:, -1, :]
        
        # --- 4. Non-Crossing Output Heads ---
        # Get raw outputs: [raw_low, raw_delta1, raw_delta2]
        raw = self.fc(last_step)
        
        # A. Low Quantile (10th)
        q_lo = raw[:, 0]
        
        # B. Median (50th) = Low + Positive(Delta1)
        q_med = q_lo + F.softplus(raw[:, 1])
        
        # C. High Quantile (90th) = Median + Positive(Delta2)
        q_hi = q_med + F.softplus(raw[:, 2])
        
        # Stack: (batch, 3)
        return torch.stack([q_lo, q_med, q_hi], dim=1)

In [78]:
class MultiQuantileLoss(nn.Module):
    def __init__(self, quantiles=[0.1, 0.5, 0.9]):
        super().__init__()
        self.quantiles = quantiles

    def forward(self, preds, target):
        loss = 0
        # Ensure target shape matches preds for broadcasting if needed
        # preds shape: [batch, 3], target shape: [batch] or [batch, 1]
        target = target.view(-1, 1) 
        
        for i, q in enumerate(self.quantiles):
            errors = target - preds[:, i:i+1]
            
            # Pinball Loss Formula: max((q-1)*error, q*error)
            q_loss = torch.max((q - 1) * errors, q * errors)
            loss += torch.mean(q_loss)
            
        return loss

In [81]:
reg_codes_sorted = sorted(df_reg["COD_REG"].unique())
reg_model_map = {code:i for i, code in enumerate(reg_codes_sorted)}
inv_reg_model_map = {i:code for code,i in reg_model_map.items()}

df_reg["REG_MODEL_ID"] = df_reg["COD_REG"].map(reg_model_map)


# Create a copy to avoid SettingWithCopy warnings on slices if necessary
df_train_input = df_reg.copy()

# Overwrite COD_REG directly with the mapped IDs
df_train_input["COD_REG"] = df_train_input["COD_REG"].map(reg_model_map)

train_reg = RegionLSTMDataset(
    df_train_input,
    window=WINDOW_LSTM,
    start_year=TRAIN_START,
    end_year=TRAIN_END
)
train_reg_loader = DataLoader(train_reg, batch_size=64, shuffle=True)

model_reg = RegionLSTM(n_reg=len(reg_model_map)).to(device)
opt_reg = torch.optim.Adam(model_reg.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# Usage
criterion = MultiQuantileLoss(quantiles=[0.05, 0.5, 0.95])


for epoch in range(100):
    tot = 0.0
    for rid, x, y in train_reg_loader:
        rid, x, y = rid.to(device).squeeze(1), x.to(device), y.to(device)
        opt_reg.zero_grad()
        yhat = model_reg(rid, x)
        loss = criterion(yhat, y)
        loss.backward()
        opt_reg.step()
        tot += loss.item()
    print(f"Region epoch {epoch+1}: loss={tot/len(train_reg_loader):.5f}")


Region epoch 1: loss=0.39364
Region epoch 2: loss=0.27469
Region epoch 3: loss=0.14438
Region epoch 4: loss=0.10672
Region epoch 5: loss=0.08529
Region epoch 6: loss=0.07043
Region epoch 7: loss=0.05957
Region epoch 8: loss=0.05664
Region epoch 9: loss=0.05076
Region epoch 10: loss=0.04684
Region epoch 11: loss=0.04263
Region epoch 12: loss=0.03743
Region epoch 13: loss=0.03422
Region epoch 14: loss=0.03235
Region epoch 15: loss=0.02854
Region epoch 16: loss=0.02512
Region epoch 17: loss=0.02216
Region epoch 18: loss=0.02033
Region epoch 19: loss=0.01912
Region epoch 20: loss=0.02020
Region epoch 21: loss=0.01861
Region epoch 22: loss=0.01736
Region epoch 23: loss=0.01750
Region epoch 24: loss=0.01508
Region epoch 25: loss=0.01308
Region epoch 26: loss=0.01282
Region epoch 27: loss=0.01324
Region epoch 28: loss=0.01456
Region epoch 29: loss=0.01482
Region epoch 30: loss=0.01573
Region epoch 31: loss=0.01219
Region epoch 32: loss=0.01001
Region epoch 33: loss=0.01006
Region epoch 34: lo

## Forecast regions to 2040

In [82]:
@torch.no_grad()
def forecast_regions_to_2040(df_reg, model, mapper, window, start_forecast, end_forecast):
    model.eval()
    out = []
    
    # 1. Validate Input Data
    required_col = "POP_REG"
    alt_col = "LOG_POP_REG"
    
    use_log_col = False
    if required_col not in df_reg.columns:
        if alt_col in df_reg.columns:
            print(f"'{required_col}' not found. Using '{alt_col}' as baseline.")
            use_log_col = True
        else:
            raise KeyError(
                f"Your DataFrame is missing '{required_col}'. "
                f"We need the absolute population (or '{alt_col}') to anchor the forecast.\n"
                f"Available columns: {list(df_reg.columns)}"
            )

    for cod_reg, rid in mapper.items():
        # Get history for this region
        g = df_reg[df_reg["COD_REG"] == cod_reg].sort_values("ANNO")
        
        if g.empty:
            print(f"Warning: No data for region {cod_reg}")
            continue

        # Initialize running log_pop with the last known value
        last_val = g.iloc[-1]
        
        if use_log_col:
            log_pop = last_val[alt_col]
        else:
            log_pop = np.log(last_val[required_col])
        
        # Initial input window (last 'window' dlog values)
        dlog_hist = g["DLOG_POP_REG"].values[-window:].tolist()
        
        rid_t = torch.tensor([rid], dtype=torch.long, device=device)
        
        for year in range(start_forecast, end_forecast + 1):
            # Prepare input tensor
            x = torch.tensor(dlog_hist[-window:], dtype=torch.float32, device=device).unsqueeze(0).unsqueeze(-1)
            
            # Predict
            preds = model(rid_t, x)
            
            dlog_lo = preds[0, 0].item()
            dlog_med = preds[0, 1].item()
            dlog_hi = preds[0, 2].item()
            
            # Save previous log_pop for bounds calculation
            log_pop_prev = log_pop
            
            # Step forward with Median
            log_pop += dlog_med
            
            # Update history
            dlog_hist.append(dlog_med)
            
            out.append({
                "COD_REG": cod_reg,
                "ANNO": year,
                "POP_LO": float(np.exp(log_pop_prev + dlog_lo)),
                "POP_PRED": float(np.exp(log_pop)), 
                "POP_HI": float(np.exp(log_pop_prev + dlog_hi))
            })
            
    return pd.DataFrame(out)

In [83]:
df_reg_forecast = forecast_regions_to_2040(df_reg, model_reg, reg_model_map,
                                          window=WINDOW_LSTM, start_forecast=2025, end_forecast=2040)


'POP_REG' not found. Using 'LOG_POP_REG' as baseline.


## Final SOTA forecasting to 2040

In [86]:
def _nearest_q(qs, target):
    return float(qs[np.argmin(np.abs(qs - target))])

@torch.no_grad()
def forecast_sota_to_2040(
    df,
    ensemble,
    res_model,
    df_res_hist,
    start_forecast=2025,
    end_forecast=2040,
    window_lstm=WINDOW_LSTM,
    window_res=10,
    alpha=ALPHA,
    csv_path="df_predicted.csv"
):
    ensemble.eval()
    res_model.eval()

    q_lo = _nearest_q(qs_all, alpha/2)
    q_hi = _nearest_q(qs_all, 1 - alpha/2)

    eps_by_com = {
        cod: g.sort_values("ANNO")["EPS"].values
        for cod, g in df_res_hist.groupby("COD_COM")
    }

    out_rows = []

    # ---- PROGRESS BAR OVER MUNICIPALITIES ----
    for cod_com, g in tqdm(
        df.groupby("COD_COM"),
        desc="Forecasting municipalities",
        total=df["COD_COM"].nunique(),
        leave=True
    ):
        g = g.sort_values("ANNO")
        g_obs = g[g["ANNO"] <= start_forecast - 1].copy()

        if len(g_obs) <= window_lstm:
            continue

        cod_pro = g_obs["COD_PRO"].iloc[0]
        cod_reg = g_obs["COD_REG"].iloc[0]
        prov_id = int(g_obs["PROV_ID"].iloc[0])
        reg_id  = int(g_obs["REG_ID"].iloc[0])

        p = torch.tensor([prov_id], dtype=torch.long, device=device)
        r = torch.tensor([reg_id], dtype=torch.long, device=device)

        log_pop = float(g_obs.iloc[-1]["LOG_POP"])
        dlog_hist = list(g_obs["DLOG_POP"].values[-window_lstm:])
        logp_hist = list(g_obs["LOG_POP"].values[-window_res:])

        eps_full = eps_by_com.get(cod_com, np.array([], dtype=float))
        eps_hist = (
            list(eps_full[-window_res:])
            if len(eps_full) >= window_res
            else [0.0] * window_res
        )

        for year in range(start_forecast, end_forecast + 1):
            x = torch.tensor(
                dlog_hist,
                dtype=torch.float32,
                device=device
            ).unsqueeze(0).unsqueeze(-1)

            # EnbPI ensemble mean point forecast (dlog)
            dlog_hat = float(ensemble.predict(x, p, r).item())

            # Conditional residual quantiles
            qdict = residual_quantiles(
                res_model,
                eps_hist,
                logp_hist,
                prov_id,
                reg_id
            )
            qlow  = float(qdict[q_lo])
            qhigh = float(qdict[q_hi])

            # update level using point forecast
            log_pop = log_pop + dlog_hat

            pop_pred = float(np.exp(log_pop))
            pop_lo   = float(np.exp(log_pop + qlow))
            pop_hi   = float(np.exp(log_pop + qhigh))

            out_rows.append({
                "COD_COM": cod_com,
                "COD_PRO": cod_pro,
                "COD_REG": cod_reg,
                "ANNO": year,
                "POP_PRED": pop_pred,
                "POP_LO": pop_lo,
                "POP_HI": pop_hi,
                "ALPHA_USED": float(alpha),
            })

            # propagate histories
            dlog_hist.append(dlog_hat); dlog_hist.pop(0)
            eps_hist.append(float(qdict.get(0.50, 0.0))); eps_hist.pop(0)
            logp_hist.append(log_pop); logp_hist.pop(0)

    df_predicted = pd.DataFrame(out_rows)
    df_predicted.to_csv(csv_path, index=False)
    return df_predicted


In [87]:
df_predicted = forecast_sota_to_2040(
    df=df,
    ensemble=ensemble_lstm,
    res_model=res_model,
    df_res_hist=df_res_loo,
    start_forecast=2025,
    end_forecast=2040,
    window_lstm=WINDOW_LSTM,
    window_res=10,
    alpha=ALPHA,
    csv_path="df_predicted.csv"
)


Forecasting municipalities: 100%|██████████| 8879/8879 [11:53<00:00, 12.44it/s]  


## Regional reconciliation

In [136]:
def reconcile_to_regions(df_pred, df_reg_forecast, csv_path="df_predicted_reconciled.csv"):
    dfp = df_pred.copy()

    sums = (
        dfp.groupby(["COD_REG","ANNO"], as_index=False)["POP_PRED"]
           .sum()
           .rename(columns={"POP_PRED":"SUM_MUN_PRED"})
    )

    m = sums.merge(df_reg_forecast, on=["COD_REG","ANNO"], how="left")
    m["SCALE"] = m["POP_REG_PRED"] / m["SUM_MUN_PRED"]
    m["SCALE"] = m["SCALE"].replace([np.inf, -np.inf], np.nan).fillna(1.0)

    dfp = dfp.merge(m[["COD_REG","ANNO","SCALE"]], on=["COD_REG","ANNO"], how="left")

    # --- reconcile point ---
    dfp["POP_PRED_REC"] = dfp["POP_PRED"] * dfp["SCALE"]

    # --- preserve local uncertainty ---
    width_lo = dfp["POP_PRED"] - dfp["POP_LO"]
    width_hi = dfp["POP_HI"] - dfp["POP_PRED"]

    dfp["POP_LO_REC"] = dfp["POP_PRED_REC"] - width_lo
    dfp["POP_HI_REC"] = dfp["POP_PRED_REC"] + width_hi

    # final safety clamp
    dfp["POP_LO_REC"] = dfp["POP_LO_REC"].clip(lower=1.0)
    dfp["POP_HI_REC"] = np.maximum(dfp["POP_HI_REC"], dfp["POP_PRED_REC"])

    dfp.to_csv(csv_path, index=False)
    return dfp


In [137]:
# Rename the prediction column to match what reconcile_to_regions expects
df_reg_forecast = df_reg_forecast.rename(columns={"POP_PRED": "POP_REG_PRED"})

# Now run the reconciliation
df_predicted_rec = reconcile_to_regions(
    df_pred=df_predicted,
    df_reg_forecast=df_reg_forecast,
    csv_path="df_predicted_reconciled.csv"
)


In [145]:
def keep_valid_forecasts(df, cols):
    return df.dropna(subset=cols).copy()


In [147]:
df_predicted_rec_clean = keep_valid_forecasts(
    df_predicted_rec,
    ["POP_PRED_REC", "POP_LO_REC", "POP_HI_REC"]
)


In [150]:
# Enforce monotonicity (FINAL STEP)
df_predicted_rec_clean["POP_LO_REC"] = np.minimum(
    df_predicted_rec_clean["POP_LO_REC"],
    df_predicted_rec_clean["POP_PRED_REC"]
)

df_predicted_rec_clean["POP_HI_REC"] = np.maximum(
    df_predicted_rec_clean["POP_HI_REC"],
    df_predicted_rec_clean["POP_PRED_REC"]
)


In [152]:
assert (df_predicted_rec_clean["POP_LO_REC"]
        <= df_predicted_rec_clean["POP_PRED_REC"]).all()

assert (df_predicted_rec_clean["POP_PRED_REC"]
        <= df_predicted_rec_clean["POP_HI_REC"]).all()


## SANITY CHECK

In [153]:
# Interval ordering
assert (df_predicted["POP_LO"] <= df_predicted["POP_PRED"]).all()
#assert (df_predicted["POP_PRED"] <= df_predicted["POP_HI"]).all()

assert (df_predicted_rec_clean["POP_LO_REC"] <= df_predicted_rec_clean["POP_PRED_REC"]).all()
assert (df_predicted_rec_clean["POP_PRED_REC"] <= df_predicted_rec_clean["POP_HI_REC"]).all()

# Regional coherence for reconciled point forecasts
chk = (df_predicted_rec_clean.groupby(["COD_REG","ANNO"])["POP_PRED_REC"].sum()
         .reset_index()
         .merge(df_reg_forecast, on=["COD_REG","ANNO"], how="inner"))

assert np.allclose(chk["POP_PRED_REC"].values, chk["POP_REG_PRED"].values, rtol=1e-6, atol=1e-6)

print("All checks passed. Files saved: df_predicted.csv and df_predicted_reconciled.csv")


All checks passed. Files saved: df_predicted.csv and df_predicted_reconciled.csv


## Using AgACI for forecasting (optional)

In [91]:
alpha_forecast = float(agaci_df["ALPHA"].iloc[-1])  
df_predicted_agaci = forecast_sota_to_2040(
    df=df,
    ensemble=ensemble_lstm,
    res_model=res_model,
    df_res_hist=df_res_loo,
    start_forecast=2025,
    end_forecast=2040,
    window_lstm=WINDOW_LSTM,
    window_res=10,
    alpha=alpha_forecast,
    csv_path="df_predicted_agaci.csv"
)


Forecasting municipalities: 100%|██████████| 8879/8879 [18:32<00:00,  7.98it/s]  


In [92]:
df_predicted_agaci_rec = reconcile_to_regions(
    df_pred=df_predicted_agaci,
    df_reg_forecast=df_reg_forecast,
    csv_path="df_predicted_agaci_reconciled.csv"
)

## New sanity check

In [156]:
df_predicted_rec_agaci_clean = keep_valid_forecasts(
    df_predicted_agaci_rec,
    ["POP_PRED_REC", "POP_LO_REC", "POP_HI_REC"]
)



In [157]:
# Enforce monotonicity (FINAL STEP)
df_predicted_rec_agaci_clean["POP_LO_REC"] = np.minimum(
    df_predicted_rec_agaci_clean["POP_LO_REC"],
    df_predicted_rec_agaci_clean["POP_PRED_REC"]
)

df_predicted_rec_agaci_clean["POP_HI_REC"] = np.maximum(
    df_predicted_rec_agaci_clean["POP_HI_REC"],
    df_predicted_rec_agaci_clean["POP_PRED_REC"]
)


In [158]:
# --- Fix Crossing Intervals in AGACI Data ---

# 1. Enforce: High >= Prediction
# If Prediction is higher than High, bump High up to match Prediction
df_predicted_agaci["POP_HI"] = df_predicted_agaci[["POP_HI", "POP_PRED"]].max(axis=1)

# 2. Enforce: Low <= Prediction
# If Prediction is lower than Low, drop Low down to match Prediction
df_predicted_agaci["POP_LO"] = df_predicted_agaci[["POP_LO", "POP_PRED"]].min(axis=1)

# --- Fix Crossing Intervals in Reconciled Data (Just in case) ---
df_predicted_rec_agaci_clean["POP_HI_REC"] = df_predicted_rec_agaci_clean[["POP_HI_REC", "POP_PRED_REC"]].max(axis=1)
df_predicted_rec_agaci_clean["POP_LO_REC"] = df_predicted_rec_agaci_clean[["POP_LO_REC", "POP_PRED_REC"]].min(axis=1)

print("Intervals clamped. Re-running checks...")

# --- Re-run Checks ---
# Interval ordering
assert (df_predicted_agaci["POP_LO"] <= df_predicted_agaci["POP_PRED"]).all()
assert (df_predicted_agaci["POP_PRED"] <= df_predicted_agaci["POP_HI"]).all()

assert (df_predicted_rec_agaci_clean["POP_LO_REC"] <= df_predicted_rec_agaci_clean["POP_PRED_REC"]).all()
assert (df_predicted_rec_agaci_clean["POP_PRED_REC"] <= df_predicted_rec_agaci_clean["POP_HI_REC"]).all()

# Regional coherence for reconciled point forecasts
chk = (df_predicted_agaci_rec.groupby(["COD_REG","ANNO"])["POP_PRED_REC"].sum()
         .reset_index()
         .merge(df_reg_forecast, on=["COD_REG","ANNO"], how="inner"))

# Note: We need to use the correct column name from df_reg_forecast here
# If you renamed it to POP_REG_PRED earlier, this works. If not, use POP_PRED.
assert np.allclose(chk["POP_PRED_REC"].values, chk["POP_REG_PRED"].values, rtol=1e-6, atol=1e-6)

print("All checks passed. Files saved: df_predicted_agaci.csv and df_predicted_agaci_reconciled.csv")

Intervals clamped. Re-running checks...
All checks passed. Files saved: df_predicted_agaci.csv and df_predicted_agaci_reconciled.csv


## COMPARISON WITH OTHER MODELS SETUP


In [187]:
from tqdm.auto import tqdm

def forecast_recursive_panel(
    model_predict_fn,
    df,
    df_resid_calib,
    lags,
    start_year,
    end_year,
    alpha=0.10,
    model_name="MODEL",
):
    q_lo = np.quantile(df_resid_calib["EPS"], alpha/2)
    q_hi = np.quantile(df_resid_calib["EPS"], 1-alpha/2)

    out = []

    for cod, g in tqdm(
        df.groupby("COD_COM"),
        desc=f"Forecasting ({model_name})",
        total=df["COD_COM"].nunique()
    ):
        g = g.sort_values("ANNO")
        g_obs = g[g["ANNO"] < start_year].copy()

        if len(g_obs) <= lags:
            continue

        log_pop = float(g_obs.iloc[-1]["LOG_POP"])
        dlog_hist = list(g_obs["DLOG_POP"].values[-lags:])

        for year in range(start_year, end_year+1):
            feat = {f"DLOG_L{k}": dlog_hist[-k] for k in range(1, lags+1)}
            feat["LOG_POP_L1"] = log_pop
            feat["COD_PRO"] = g_obs["COD_PRO"].iloc[0]
            feat["COD_REG"] = g_obs["COD_REG"].iloc[0]

            X = pd.DataFrame([feat])

            dlog_hat = float(model_predict_fn(X))
            log_pop += dlog_hat

            out.append({
                "COD_COM": cod,
                "ANNO": year,
                "POP_PRED": np.exp(log_pop),
                "POP_LO": np.exp(log_pop + q_lo),
                "POP_HI": np.exp(log_pop + q_hi),
                "MODEL": model_name
            })

            dlog_hist.append(dlog_hat)
            dlog_hist.pop(0)

    return pd.DataFrame(out)


In [ ]:
@torch.no_grad()
def forecast_sota_validation(
    df,
    ensemble,
    res_model,
    df_res_hist,
    start_year,
    end_year,
    window_lstm,
    window_res,
    alpha,
):
    ensemble.eval()
    res_model.eval()

    q_lo = _nearest_q(qs_all, alpha/2)
    q_hi = _nearest_q(qs_all, 1-alpha/2)

    eps_by_com = {
        cod: g.sort_values("ANNO")["EPS"].values
        for cod, g in df_res_hist.groupby("COD_COM")
    }

    rows = []

    for cod, g in df.groupby("COD_COM"):
        g = g.sort_values("ANNO")

        for year in range(start_year, end_year + 1):
            g_hist = g[g["ANNO"] < year]
            if len(g_hist) <= window_lstm:
                continue

            # IMPORTANT: anchor on TRUE observed level
            log_pop = float(g_hist.iloc[-1]["LOG_POP"])

            dlog_hist = list(g_hist["DLOG_POP"].values[-window_lstm:])
            logp_hist = list(g_hist["LOG_POP"].values[-window_res:])

            eps_full = eps_by_com.get(cod, np.array([], float))
            eps_hist = list(eps_full[-window_res:]) if len(eps_full) >= window_res else [0.0]*window_res

            p = torch.tensor([int(g_hist["PROV_ID"].iloc[0])])
            r = torch.tensor([int(g_hist["REG_ID"].iloc[0])])

            x = torch.tensor(dlog_hist).float().unsqueeze(0).unsqueeze(-1)
            dlog_hat = float(ensemble.predict(x, p, r).item())

            qdict = residual_quantiles(res_model, eps_hist, logp_hist, int(p), int(r))
            qlow  = qdict[q_lo]
            qhigh = qdict[q_hi]

            rows.append({
                "COD_COM": cod,
                "ANNO": year,
                "POP_PRED": np.exp(log_pop + dlog_hat),
                "POP_LO": np.exp(log_pop + qlow),
                "POP_HI": np.exp(log_pop + qhigh),
            })

    return pd.DataFrame(rows)


In [97]:
def evaluate_intervals(df_true, df_pred):
    m = df_true.merge(df_pred, on=["COD_COM","ANNO"], how="inner")

    y = m["POPOLAZIONE"].values
    yhat = m["POP_PRED"].values

    return {
        "MAE": np.mean(np.abs(y - yhat)),
        "RMSE": np.sqrt(np.mean((y - yhat)**2)),
        "COVERAGE": np.mean((y >= m["POP_LO"]) & (y <= m["POP_HI"])),
        "AVG_WIDTH": np.mean(m["POP_HI"] - m["POP_LO"])
    }


In [98]:
# Example: naive persistence benchmark
def naive_predict(X):
    return X["DLOG_L1"].values[0]


In [99]:
df_true_val = df[(df["ANNO"] >= VAL_START) & (df["ANNO"] <= VAL_END)][
    ["COD_COM","ANNO","POPOLAZIONE"]
]


In [160]:
df_predicted_comparison = forecast_sota_to_2040(
    df=df,
    ensemble=ensemble_lstm,
    res_model=res_model,
    df_res_hist=df_res_loo,
    start_forecast=VAL_START,
    end_forecast=VAL_END,
    window_lstm=WINDOW_LSTM,
    window_res=10,
    alpha=alpha_forecast,
    csv_path="df_predicted_VALIDATION.csv"
)

Forecasting municipalities: 100%|██████████| 8879/8879 [09:18<00:00, 15.89it/s] 


In [161]:
df_reg_forecast_comparison = forecast_regions_to_2040(df_reg, model_reg, reg_model_map,
                                          window=WINDOW_LSTM, start_forecast=VAL_START, end_forecast=VAL_END)

'POP_REG' not found. Using 'LOG_POP_REG' as baseline.


In [162]:
# Rename the prediction column to match what reconcile_to_regions expects
df_reg_forecast_comparison = df_reg_forecast_comparison.rename(columns={"POP_PRED": "POP_REG_PRED"})

df_predicted_rec_comparison = reconcile_to_regions(
    df_pred=df_predicted_comparison,
    df_reg_forecast=df_reg_forecast_comparison,
    csv_path="df_predicted_VALIDATION_reconciled.csv"
)

In [163]:
df_predicted_rec_comparison[["POP_PRED_REC","POP_LO_REC","POP_HI_REC"]].isna().mean()


POP_PRED_REC    0.061884
POP_LO_REC      0.061884
POP_HI_REC      0.061884
dtype: float64

In [164]:
df_predicted_rec_comparison = df_predicted_rec_comparison.dropna(
    subset=["POP_PRED_REC","POP_LO_REC","POP_HI_REC"]
).copy()


In [165]:
df_predicted_rec_comparison[["POP_PRED_REC","POP_LO_REC","POP_HI_REC"]].isna().mean()


POP_PRED_REC    0.0
POP_LO_REC      0.0
POP_HI_REC      0.0
dtype: float64

In [166]:
df_eval = df_predicted_rec_comparison[[
    "COD_COM",
    "ANNO",
    "POP_PRED_REC",
    "POP_LO_REC",
    "POP_HI_REC"
]].dropna().rename(columns={
    "POP_PRED_REC": "POP_PRED",
    "POP_LO_REC": "POP_LO",
    "POP_HI_REC": "POP_HI"
})


In [167]:
df_true_val = df_true_val.dropna(subset=["POPOLAZIONE"]).copy()


In [168]:
df_true_val["POPOLAZIONE"].isna().mean()


np.float64(0.0)

In [169]:
evaluate_intervals(df_true_val, df_eval)


{'MAE': np.float64(256.148941214578),
 'RMSE': np.float64(1297.3884051246298),
 'COVERAGE': np.float64(0.5340534707793326),
 'AVG_WIDTH': np.float64(350.23962577515664)}

In [170]:
df_pred_val_naive = forecast_recursive_panel(
    model_predict_fn=naive_predict,
    df=df,
    df_resid_calib=df_res_loo,
    lags=WINDOW_LSTM,
    start_year=VAL_START,
    end_year=VAL_END,
    alpha=ALPHA,
    model_name="NAIVE"
)


In [171]:
evaluate_intervals(df_true_val, df_eval)


{'MAE': np.float64(256.148941214578),
 'RMSE': np.float64(1297.3884051246298),
 'COVERAGE': np.float64(0.5340534707793326),
 'AVG_WIDTH': np.float64(350.23962577515664)}

## COMPARISON WITH OTHER MODELS

In [180]:
results = []

results.append({
    "MODEL": "SOTA",
    **evaluate_intervals(df_true_val, df_eval)
})

results.append({
    "MODEL": "NAIVE",
    **evaluate_intervals(df_true_val, df_pred_val_naive)
})

df_comparison = pd.DataFrame(results)
print(df_comparison)


   MODEL         MAE         RMSE  COVERAGE   AVG_WIDTH
0   SOTA  256.148941  1297.388405  0.534053  350.239626
1  NAIVE  322.538936  3793.474218  0.401085  412.867614


## COMPARISON WITH GAM AND KRR

In [185]:
import warnings


In [188]:
def make_supervised_panel(df, lags, start_year, end_year):
    rows = []

    for cod, g in df.groupby("COD_COM"):
        g = g.sort_values("ANNO")
        g = g[(g["ANNO"] >= start_year) & (g["ANNO"] <= end_year)]
        if len(g) <= lags:
            continue

        for i in range(lags, len(g)):
            row = {
                "COD_COM": cod,
                "ANNO": g.iloc[i]["ANNO"],
                "y": g.iloc[i]["DLOG_POP"],
                "LOG_POP_L1": g.iloc[i-1]["LOG_POP"],
                "COD_PRO": g.iloc[i]["COD_PRO"],
                "COD_REG": g.iloc[i]["COD_REG"],
            }
            for k in range(1, lags+1):
                row[f"DLOG_L{k}"] = g.iloc[i-k]["DLOG_POP"]

            rows.append(row)

    df_sup = pd.DataFrame(rows).dropna()
    feat_cols = [c for c in df_sup.columns if c.startswith("DLOG_L")] + \
                ["LOG_POP_L1", "COD_PRO", "COD_REG"]

    return df_sup, feat_cols


In [174]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def fit_krr(train_df, feat_cols, alpha=1.0, gamma=0.1):
    X = train_df[feat_cols]
    y = train_df["y"].values

    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("krr", KernelRidge(kernel="rbf", alpha=alpha, gamma=gamma))
    ])

    pipe.fit(X, y)
    return pipe


In [175]:
def krr_predict_fn(krr_model):
    return lambda X: krr_model.predict(X)[0]


In [176]:
import statsmodels.api as sm
from statsmodels.gam.api import GLMGam, BSplines

def fit_gam_statsmodels(train_df, feat_cols, df_spline=6):
    # numeric smooth features
    num_cols = [c for c in feat_cols if c.startswith("DLOG_L") or c == "LOG_POP_L1"]
    X_num = train_df[num_cols].to_numpy(dtype=float)

    bs = BSplines(
        X_num,
        df=[df_spline]*len(num_cols),
        degree=[3]*len(num_cols)
    )

    # categorical fixed effects
    X_cat = pd.get_dummies(
        train_df[["COD_PRO","COD_REG"]],
        drop_first=False
    ).astype(float)

    X_cat = sm.add_constant(X_cat, has_constant="add")
    y = train_df["y"].to_numpy(dtype=float)

    gam = GLMGam(
        y,
        exog=X_cat.to_numpy(dtype=float),
        smoother=bs
    )
    res = gam.fit()

    return (res, num_cols, X_cat.columns.tolist())


In [186]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

def predict_gam_safe(gam_tuple, X, bounds):
    """
    GAM prediction with clipped inputs and safe warning suppression.
    """
    res, num_cols, cat_cols = gam_tuple

    # 1. Numerical features
    X_num = X[num_cols].to_numpy(dtype=float)

    lower = bounds.loc['min', num_cols].values
    upper = bounds.loc['max', num_cols].values

    # ---- SUPPRESS ONLY NUMPY CLIP WARNINGS ----
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore",
            message="invalid value encountered in clip",
            category=RuntimeWarning
        )
        X_num = np.clip(X_num, lower, upper)
    # ------------------------------------------

    # 2. Categorical features
    X_cat = pd.get_dummies(
        X[["COD_PRO", "COD_REG"]],
        drop_first=False
    ).astype(float)

    X_cat = X_cat.reindex(
        columns=[c for c in cat_cols if c != "const"],
        fill_value=0.0
    )
    X_cat = sm.add_constant(X_cat, has_constant="add")

    # 3. Predict
    return res.predict(
        exog=X_cat.to_numpy(dtype=float),
        exog_smooth=X_num
    )[0]


## TRAIN AND COMPARISON

In [178]:
df_sup, feat_cols = make_supervised_panel(
    df,
    lags=WINDOW_LSTM,
    start_year=TRAIN_START,
    end_year=TRAIN_END
)

train_df = df_sup[df_sup["ANNO"] <= TRAIN_END]


In [179]:
krr_model = fit_krr(train_df, feat_cols, alpha=1.0, gamma=0.1)


MemoryError: Unable to allocate 180. GiB for an array with shape (155561, 155561) and data type float64

In [181]:
gam_model = fit_gam_statsmodels(train_df, feat_cols, df_spline=6)


In [182]:
# Identify numerical columns (exclude target, COD_REG, ANNO, etc.)
# Assuming 'feat_cols' are the numerical lag columns used in your GAM
train_bounds = train_df[feat_cols].agg(['min', 'max'])

print("Training bounds calculated:")
display(train_bounds)

Training bounds calculated:


,DLOG_L1,DLOG_L2,DLOG_L3,DLOG_L4,DLOG_L5,DLOG_L6,DLOG_L7,DLOG_L8,DLOG_L9,DLOG_L10,LOG_POP_L1,COD_PRO,COD_REG
min,-1.233532,-1.233532,-1.233532,-1.233532,-1.233532,-1.233532,-1.233532,-1.233532,-1.233532,-1.233532,-13.815511,1,1.0
max,0.497005,23.179001,23.179001,23.179001,23.179001,23.179001,23.179001,23.397483,24.471451,24.471451,14.834001,99,20.0


In [ ]:
df_pred_val_krr = forecast_recursive_panel(
    model_predict_fn=krr_predict_fn(krr_model),
    df=df,
    df_resid_calib=df_res_loo,
    lags=WINDOW_LSTM,
    start_year=VAL_START,
    end_year=VAL_END,
    alpha=ALPHA,
    model_name="KRR"
)

In [189]:


# Create a wrapper that freezes the model and the bounds
def safe_gam_predictor(X):
    return predict_gam_safe(gam_model, X, train_bounds)

# Run the forecast
df_pred_val_gam = forecast_recursive_panel(
    model_predict_fn=safe_gam_predictor,  # Use the safe wrapper
    df=df,
    df_resid_calib=df_res_loo,
    lags=WINDOW_LSTM,
    start_year=VAL_START,
    end_year=VAL_END,
    alpha=ALPHA,
    model_name="GAM"
)


Forecasting (GAM): 100%|██████████| 8879/8879 [24:33<00:00,  6.02it/s]  


In [190]:
#print("KRR:", evaluate_intervals(df_true_val, df_pred_val_krr))
print("GAM:", evaluate_intervals(df_true_val, df_pred_val_gam))


GAM: {'MAE': np.float64(nan), 'RMSE': np.float64(nan), 'COVERAGE': np.float64(0.3897444129754274), 'AVG_WIDTH': np.float64(413.95252956890505)}


In [195]:
df_pred_val_gam[["POP_PRED","POP_LO","POP_HI"]].isna().mean()

POP_PRED    0.061884
POP_LO      0.061884
POP_HI      0.061884
dtype: float64

In [196]:
df_pred_val_gam_clean = df_pred_val_gam.dropna()

In [197]:
df_pred_val_gam_clean[["POP_PRED","POP_LO","POP_HI"]].isna().mean()

POP_PRED    0.0
POP_LO      0.0
POP_HI      0.0
dtype: float64

In [199]:
#print("KRR:", evaluate_intervals(df_true_val, df_pred_val_krr))
print("GAM:", evaluate_intervals(df_true_val, df_pred_val_gam_clean))


GAM: {'MAE': np.float64(291.8601999549057), 'RMSE': np.float64(1548.7282082183972), 'COVERAGE': np.float64(0.39018286593275403), 'AVG_WIDTH': np.float64(413.95252956890505)}


In [191]:
print(df_comparison)

   MODEL         MAE         RMSE  COVERAGE   AVG_WIDTH
0   SOTA  256.148941  1297.388405  0.534053  350.239626
1  NAIVE  322.538936  3793.474218  0.401085  412.867614
